In [1]:
targets = []
with open("aso-candidates-21mer.txt") as f:
    for l in f:
        l = l.strip()
        if l:
            targets.append(l)
print("Loaded %d targets" % len(targets))


Loaded 1650 targets


In [2]:
import pyensembl 
human = pyensembl.genome_for_reference_name("GRCh38")
mouse = pyensembl.genome_for_reference_name("GRCm38")
human_transcripts = human.transcript_sequences.fasta_dictionary
mouse_transcripts = mouse.transcript_sequences.fasta_dictionary
human_nutm1 = human.genes_by_name("NUTM1")[0]
mouse_nutm1 = mouse.genes_by_name("Nutm1")[0]
human_nutm1_transcripts = human_nutm1.transcripts
human_nutm1_transcript_ids = {t.id for t in human_nutm1_transcripts}
mouse_nutm1_transcripts = mouse_nutm1.transcripts
mouse_nutm1_transcript_ids = {t.id for t in mouse_nutm1_transcripts}


INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/iskander/Library/Caches/pyensembl/GRCh38/ensembl109/Homo_sapiens.GRCh38.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/iskander/Library/Caches/pyensembl/GRCh38/ensembl109/Homo_sapiens.GRCh38.ncrna.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/iskander/Library/Caches/pyensembl/GRCm38/ensembl102/Mus_musculus.GRCm38.cdna.all.fa.gz.pickle
INFO:pyensembl.sequence_data:Loaded sequence dictionary from /Users/iskander/Library/Caches/pyensembl/GRCm38/ensembl102/Mus_musculus.GRCm38.ncrna.fa.gz.pickle


In [40]:

human_nutm1_seqs = "#".join({v for (k, v) in human_transcripts.items() if k in human_nutm1_transcript_ids})
print(human_nutm1_seqs.count("#") + 1, len(human_nutm1_seqs))

human_other_seqs = "#".join({v for (k, v) in human_transcripts.items() if k not in human_nutm1_transcript_ids})
print(str((human_other_seqs.count("#") + 1) // 1000) + "k", str(len(human_other_seqs) // 10**5 / 10) + "M")
 
mouse_nutm1_seqs = "#".join({v for (k, v) in mouse_transcripts.items() if k in mouse_nutm1_transcript_ids})
print(mouse_nutm1_seqs.count("#") + 1, len(mouse_nutm1_seqs))

mouse_other_seqs = "#".join({v for (k, v) in mouse_transcripts.items() if k not in mouse_nutm1_transcript_ids})
print(str((mouse_other_seqs.count("#") + 1) // 1000) + "k", str(len(mouse_other_seqs) //10 ** 5 / 10) + "M" )


4 16025
259k 448.7M
1 3731
137k 234.9M


In [43]:
from tqdm import tqdm
import pandas as pd

from sys import intern

from collections import Counter, defaultdict

def revcomp(seq):
    d = {"A": "T", "T": "A", "C": "G", "G": "C"}
    return "".join([d[c] for c in seq][::-1])

def get_or_generate_kmers(tid, seq, k, _cache={}):
    key = (tid, k)
    if key not in _cache:  
        _cache[key] = {intern(seq[i:i+k]) for i in range(len(seq) - k + 1)}
    return _cache[key]

def annotate(targets):
    if type(targets) is str:
        targets = [targets]
    
    rows = []
    ks = sorted({14}.union({len(target) for target in targets}))
    all_kmers = {
        target[i:i+k]
        for target in targets
        for k in ks
        for i in range(len(target) - k + 1)
    }
    print("%d unique kmers of size %s across %d targets" % (
        len(all_kmers),
        ks,
        len(targets)
    ))
    print("...counting kmer occurrences")
    kmer_counts = defaultdict(dict)

    for kmer in tqdm(all_kmers):
        c = kmer_counts[kmer]
        c["human_nutm1"] = human_nutm1_seqs.count(kmer)
        c["human_other"] = human_other_seqs.count(kmer)
        c["mouse_nutm1"] = mouse_nutm1_seqs.count(kmer)
        c["mouse_other"] = mouse_other_seqs.count(kmer)

    for target in targets:
        print(target)
        

        n = len(target)
        features = {
            "target": target,
            "len": n,
            "A_count": 0, 
            "C_count": 0, 
            "G_count": 0, 
            "T_count": 0, 
            "GC_count": 0, 
            "AT_count": 0,
            "longest_homopolymer": 0,

        }


        longest_homopolymer = 0
        last_nt = ""
        current_homopolymer = 0

        for i, nt in enumerate(target):
            features["%s_count" % nt] += 1
            is_gc = nt == "G" or nt == "C"
            features["GC_count"] += is_gc
            features["AT_count"] += not is_gc
            if last_nt == nt:
                current_homopolymer += 1
            if last_nt != nt or i == n - 1:
                last_nt = nt
                longest_homopolymer = max(longest_homopolymer, current_homopolymer)
                current_homopolymer = 1

        features["longest_homopolymer"] = longest_homopolymer

        
        for k in ks:
            features["human_on_target_transcript_count_%dmer" % k] = 0
            features["human_off_target_transcript_count_%dmer" % k] = 0
            features["mouse_on_target_transcript_count_%dmer" % k] = 0
            features["mouse_off_target_transcript_count_%dmer" % k] = 0
            
            print("=== searching for %dmers" % k)
            for i in tqdm(range(n - k + 1)):
                kmer = target[i:i+k]
                c = kmer_counts[kmer]
                features["human_on_target_transcript_count_%dmer" % k] += c["human_nutm1"]
                features["human_off_target_transcript_count_%dmer" % k] += c["human_other"]
                features["mouse_on_target_transcript_count_%dmer" % k] += c["mouse_nutm1"]
                features["mouse_off_target_transcript_count_%dmer" % k] += c["mouse_other"]
        rows.append(features)
    df = pd.DataFrame(rows)
    df['revcomp_target'] = [revcomp(s) for s in df.target]
    return df


In [44]:
annotate("CCGGCGTCAGCGTAAAGCCCA")

9 unique kmers of size [14, 21] across 1 targets
...counting kmer occurrences


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:25<00:00,  2.87s/it]


CCGGCGTCAGCGTAAAGCCCA
=== searching for 14mers


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 103883.69it/s]


=== searching for 21mers


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 27962.03it/s]


,target,len,A_count,C_count,G_count,T_count,GC_count,AT_count,longest_homopolymer,human_on_target_transcript_count_14mer,human_off_target_transcript_count_14mer,mouse_on_target_transcript_count_14mer,mouse_off_target_transcript_count_14mer,human_on_target_transcript_count_21mer,human_off_target_transcript_count_21mer,mouse_on_target_transcript_count_21mer,mouse_off_target_transcript_count_21mer,revcomp_target
0,CCGGCGTCAGCGTAAAGCCCA,21,5,8,6,2,14,7,3,32,0,0,0,4,0,0,0,TGGGCTTTACGCTGACGCCGG


In [45]:
df = annotate(targets)


3307 unique kmers of size [14, 21] across 1650 targets
...counting kmer occurrences


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3307/3307 [6:05:55<00:00,  6.64s/it]


AGCCCAGGAGTTTGGCTGAGC
=== searching for 14mers


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 4069.67it/s]


=== searching for 21mers


...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 47662.55it/s]


GTTATGATCAGAATCCTTCCC
=== searching for 14mers


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 228261.44it/s]


=== searching for 21mers


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [47]:
len(df)

1650

In [53]:
t = human.transcripts_by_name("NUTM1-201")[0]
df["NUTM1-201-position"] = [t.sequence.index(s) for s in df.target]

In [77]:
chosen = \
    df[ (df.AT_count >= 6) &
        (df.longest_homopolymer < 3) & 
        (df.human_off_target_transcript_count_14mer == 0) & 
       (df.mouse_off_target_transcript_count_14mer < 10) & 
       (df.AT_count > 7) & 
       (df.revcomp_target.str.startswith(("C", "G")))
       
    ]
len(chosen)

3

In [78]:
chosen

,target,len,A_count,C_count,G_count,T_count,GC_count,AT_count,longest_homopolymer,human_on_target_transcript_count_14mer,human_off_target_transcript_count_14mer,mouse_on_target_transcript_count_14mer,mouse_off_target_transcript_count_14mer,human_on_target_transcript_count_21mer,human_off_target_transcript_count_21mer,mouse_on_target_transcript_count_21mer,mouse_off_target_transcript_count_21mer,revcomp_target,NUTM1-201-position
557,CAGAAGCGACTCATGGCCTTG,21,5,6,6,4,12,9,2,32,0,0,7,4,0,0,0,CAAGGCCATGAGTCGCTTCTG,1556
1047,TTCAGTGGCGCTCAGTTGGAC,21,3,5,7,6,12,9,2,32,0,0,5,4,0,0,0,GTCCAACTGAGCGCCACTGAA,1610
1526,ATGGCCATCTAGGAGGCGCTG,21,4,5,8,4,13,8,2,32,0,0,1,4,0,0,0,CAGCGCCTCCTAGATGGCCAT,1947


In [91]:
import numpy as np
random_indices = np.arange(21)
np.random.seed(42)
for  s, loc in zip(chosen["revcomp_target"], chosen["NUTM1-201-position"]):
    np.random.shuffle(random_indices)
    print("NUTM1-201:%d\n\tTarget=%s\n\tScrambled=%s" % (
        loc + 1, s, "".join([s[i] for i in random_indices])))

NUTM1-201:1557
	Target=CAAGGCCATGAGTCGCTTCTG
	Scrambled=CTCATCGGCTCAGGGTAAGTC
NUTM1-201:1611
	Target=GTCCAACTGAGCGCCACTGAA
	Scrambled=CAGAGAACTCAAGCGCTCCGT
NUTM1-201:1948
	Target=CAGCGCCTCCTAGATGGCCAT
	Scrambled=TGGCCCCTCGACGCATGCTAA


In [99]:
# IDT bulk order
import numpy as np
random_indices = np.arange(21)
np.random.seed(42)
for  aso_idx, (s, loc) in enumerate(zip(chosen["revcomp_target"], chosen["NUTM1-201-position"])):
    np.random.shuffle(random_indices)
    print("ASO%d-NUTM1-201-%d\t%s\t200asocustom" % (aso_idx + 1, loc + 1, "*".join(s)))
    print("ASO%ds-NUTM1-201-%d-Scrambled\t%s\t200asocustom" % (
        aso_idx + 1, loc + 1,  "*".join([s[i] for i in random_indices])))

ASO1-NUTM1-201-1557	C*A*A*G*G*C*C*A*T*G*A*G*T*C*G*C*T*T*C*T*G	200asocustom
ASO1s-NUTM1-201-1557-Scrambled	C*T*C*A*T*C*G*G*C*T*C*A*G*G*G*T*A*A*G*T*C	200asocustom
ASO2-NUTM1-201-1611	G*T*C*C*A*A*C*T*G*A*G*C*G*C*C*A*C*T*G*A*A	200asocustom
ASO2s-NUTM1-201-1611-Scrambled	C*A*G*A*G*A*A*C*T*C*A*A*G*C*G*C*T*C*C*G*T	200asocustom
ASO3-NUTM1-201-1948	C*A*G*C*G*C*C*T*C*C*T*A*G*A*T*G*G*C*C*A*T	200asocustom
ASO3s-NUTM1-201-1948-Scrambled	T*G*G*C*C*C*C*T*C*G*A*C*G*C*A*T*G*C*T*A*A	200asocustom


In [82]:
chosen.seq_reverse_complement

9     TCTGGGCTTTACGCTGACGCC
18    GGGCTTTACGCTGACGCCGGC
43    CTCTGGGCTTTACGCTGACGC
44    GGCTTTACGCTGACGCCGGCG
45    CTGGGCTTTACGCTGACGCCG
47    TGGGCTTTACGCTGACGCCGG
Name: seq_reverse_complement, dtype: object

In [83]:
df

,seq,len,A_count,C_count,G_count,T_count,GC_count,AT_count,longest_homopolymer,human_on_target_transcript_count,human_off_target_transcript_count,human_on_target_transcript_count_14mer,human_off_target_transcript_count_14mer,mouse_on_target_transcript_count_14mer,mouse_off_target_transcript_count_14mer,human_on_target_transcript_count_21mer,human_off_target_transcript_count_21mer,mouse_on_target_transcript_count_21mer,mouse_off_target_transcript_count_21mer,seq_reverse_complement
0,GTGAAGGAGTATGTTGACATC,21,6,2,7,6,9,12,1,0,0,32,38,0,13,4,0,0,0,GATGTCAACATACTCCTTCAC
1,TGCTTCGTTCCCTGGCCCGGC,21,0,9,6,6,15,6,2,0,0,32,24,6,11,4,0,0,0,GCCGGGCCAGGGAACGAAGCA
2,TTGGCTGTGCAGGAGTGGGAG,21,3,2,11,5,13,8,2,0,0,32,75,0,31,4,0,0,0,CTCCCACTCCTGCACAGCCAA
3,GAGTATGTTGACATCATGGAA,21,7,2,6,6,8,13,1,0,0,32,11,4,11,4,0,0,0,TTCCATGATGTCAACATACTC
4,GTGCTTCGTTCCCTGGCCCGG,21,0,8,7,6,15,6,2,0,0,32,23,7,7,4,0,0,0,CCGGGCCAGGGAACGAAGCAC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,GCTTCGTTCCCTGGCCCGGCT,21,0,9,6,6,15,6,2,0,0,32,35,5,14,4,0,0,0,AGCCGGGCCAGGGAACGAAGC
61,TGAAGGAGTATGTTGACATCA,21,7,2,6,6,8,13,1,0,0,32,34,0,7,4,0,0,0,TGATGTCAACATACTCCTTCA
62,TGCAACCCCTTTGAAACTTGA,21,6,6,3,6,9,12,3,0,0,32,28,3,3,4,0,0,0,TCAAGTTTCAAAGGGGTTGCA
63,CCCCGCCGGCGTCAGCGTAAA,21,4,9,6,2,15,6,3,0,0,32,2,0,0,4,0,0,0,TTTACGCTGACGCCGGCGGGG


In [91]:
chosen.seq_reverse_complement

9     TCTGGGCTTTACGCTGACGCC
18    GGGCTTTACGCTGACGCCGGC
43    CTCTGGGCTTTACGCTGACGC
44    GGCTTTACGCTGACGCCGGCG
45    CTGGGCTTTACGCTGACGCCG
47    TGGGCTTTACGCTGACGCCGG
Name: seq_reverse_complement, dtype: object

In [101]:
human_nutm1_seqs.index("CAAGGCCATGAGTCGCTTCTG")

ValueError: substring not found